# Load Data

In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize

from sklearn import metrics

Using TensorFlow backend.


In [0]:
from IPython.display import clear_output
# Import necessary library
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

path = "/content/drive/My Drive/CA683 Data Mining/Continuous Assignment/InsideAirbnb/data/"
calendar = pd.read_csv(path+"calendar.csv")
listings = pd.read_csv(path+"listings.csv")
reviews = pd.read_csv(path+"reviews.csv")
clear_output()

In [0]:
col_list = list(listings.columns)
# basicly the text information about the host
text_columns=['summary','space','description',"neighborhood_overview","notes","transit","access","interaction","house_rules","host_about"]
# 
host_string_columns = ["host_name","host_since","amenities"]

host_location_columns = ["host_neighbourhood","street","neighbourhood","neighbourhood_cleansed","city","market","host_location",
                         "latitude","longitude"]

host_numeric_columns = ["host_id","host_acceptance_rate","host_response_time","host_response_rate","host_listings_count",
                        "host_total_listings_count","square_feet","price","weekly_price","monthly_price",
                        "security_deposit","cleaning_fee","guests_included","extra_people","number_of_reviews",
                        "number_of_reviews_ltm","calculated_host_listings_count","calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
                        "reviews_per_month"]

host_date_columns = ["first_review"]

host_nights_columns = col_list[67:75]

host_category_columns = ["host_is_superhost","host_has_profile_pic","host_identity_verified","property_type","room_type",
                         "instant_bookable","is_business_travel_ready","require_guest_profile_picture","require_guest_phone_verification"]

host_score_columns = ["review_scores_rating","review_scores_accuracy","review_scores_cleanliness","review_scores_checkin",
                      "review_scores_communication","review_scores_location","review_scores_value"
                      ]
host_other_columns = ["host_verifications",]

# Basic Model on Summary

In [0]:
listings.head(1)

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,...,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,44077,https://www.airbnb.com/rooms/44077,20200321154917,2020-03-21,cosy comfortable Irish cottage twin,Our house was built in 1937 when there was ple...,"Hi, My name is Teresa and we have 3 rooms ava...",Our house was built in 1937 when there was ple...,none,I like our neighbourhood as there is no shorta...,Dundrum was always known as a Spa Town and the...,There is a bus stop 2 mins up the road from us...,We have a beautiful landscaped garden with tab...,We live on the premises so we are always on ha...,Welcome to our cottage and we hope you will en...,NaN,NaN,https://a0.muscache.com/im/pictures/525706/050...,NaN,193005,https://www.airbnb.com/users/show/193005,Teresa,2010-08-06,Dublin Ireland,"I am a friendly outgoing Irishwoman ,who love...",within an hour,100%,100%,t,https://a0.muscache.com/im/users/193005/profil...,https://a0.muscache.com/im/users/193005/profil...,Churchtown,3.0,3.0,"['email', 'phone', 'facebook', 'reviews', 'jum...",t,f,"Churchtown, Ireland",Churchtown,Dn Laoghaire-Rathdown,...,$0.00,3,1125,3,3,1125,1125,3.0,1125.0,a week ago,t,7,11,23,275,2020-03-21,227,38,2011-03-20,2020-03-06,95.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,NaN,t,f,moderate,t,t,3,0,3,0,2.07


In [0]:
df = listings[['summary']]

In [0]:
listings[host_score_columns].head(2)

,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,95.0,10.0,10.0,10.0,10.0,10.0,10.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#@title Drop missing value
nan_list = df[df['score'].isnull()].index.tolist()
print(len(nan_list))
df = df.drop(nan_list)
df.head(3)

1653


,summary,score
0,Our house was built in 1937 when there was ple...,100.000000
2,Our Cottage is a charming light filled cottage...,98.333333
3,Welcoming victorian house. Breakfast included....,91.666667


In [0]:
# calculate overall score
res = listings['review_scores_accuracy']
for target in host_score_columns[2:]:
  print(target)
  res = listings[target]+res
df['score'] = (res/6)*10
df['score']= df['score'].astype(int)
clear_output()
df.head(3)

,summary,score
0,Our house was built in 1937 when there was ple...,100
2,Our Cottage is a charming light filled cottage...,98
3,Welcoming victorian house. Breakfast included....,91


# Features

In [0]:
text = df['summary'].iloc[0]
type(text)

str

In [0]:
#@title CountVectorizer example
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=0,lowercase=0)
vectorizer.fit([text])
vectorizer.transform([text]).toarray()

array([[1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2,
        1, 1, 1, 1, 2, 1, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 4, 1, 1, 1, 3, 4, 1, 1,
        1]])

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vector = TfidfVectorizer(analyzer='word',token_pattern='\w{1,}',max_features=5000)
tfidf_vector.fit([text])
tfidf_vector.transform([text]).toarray()

array([[0.0745356, 0.0745356, 0.0745356, 0.2236068, 0.1490712, 0.1490712,
        0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356,
        0.1490712, 0.0745356, 0.0745356, 0.1490712, 0.0745356, 0.0745356,
        0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.1490712,
        0.0745356, 0.2236068, 0.2236068, 0.1490712, 0.0745356, 0.0745356,
        0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.1490712,
        0.1490712, 0.0745356, 0.2236068, 0.0745356, 0.0745356, 0.0745356,
        0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356, 0.0745356,
        0.0745356, 0.0745356, 0.0745356, 0.2236068, 0.0745356, 0.0745356,
        0.0745356, 0.0745356, 0.2981424, 0.0745356, 0.0745356, 0.0745356,
        0.2236068, 0.372678 , 0.1490712, 0.0745356, 0.0745356, 0.0745356]])

In [0]:
sentences = df['summary'].fillna('').values
y = df['score']
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences,y)
vectorizer.fit(sentences)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)

In [0]:
vectorizer.fit(sentences)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=0, max_df=1.0, max_features=None, min_df=0,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
X_train_tfidf = tfidf_vector.fit(X)
X_train_tfidf = vectorizer.transform(sentences_train)
X_test_tfidf  = vectorizer.transform(sentences_test)

## BayesianRidge

In [0]:
from sklearn.linear_model import BayesianRidge
model = BayesianRidge()
model.fit(X_train.toarray(),y_train)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)

In [0]:
y_pred = model.predict(X_test.toarray())

In [0]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 3.765504519540523
Mean Squared Error: 31.273803588985032
Root Mean Squared Error: 5.592298596193253


In [0]:
from sklearn.linear_model import LinearRegression
clf = LogisticRegression()
clf.fit(X_train.toarray(),y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Plan
## Feature Extraction from text
+ Count Vectors as features
+ TF-IDF Vectors as features
  1. Word Level TF-IDF : Matrix representing tf-idf scores of every term in different documents
  2. N-gram Level TF-IDF : N-grams are the combination of N terms together. This Matrix representing tf-idf scores of N-grams
  3. Character Level TF-IDF : Matrix representing tf-idf scores of character level n-grams in the corpus

+ Word Embeddings
+ Text / NLP based features
+ Topic Models as features

## Modeling 
+ Classification
  + Naive Bayes Classifier
  + Linear Classifier
  + Support Vector Machine
+ Regression
 + Linear Regression
 + Ridge Regression
 + Lasso Regression
 + Polynomial Regression
 + ElasticNet Regression
 + randomForest
 + Support Vector Machine
  

## [Evaluation](https://machinelearningmastery.com/metrics-evaluate-machine-learning-algorithms-python/)
+ Classification
  + Classification Accuracy
  + Log Loss
  + Area Under ROC Curve
  + Confusion Matrix
  + Classification Report
+ Regression
  + Mean Absolute Error
  + Mean Squared Error
  + R^2 Metric



In [0]:
df_sample = df.sample(1000)
sentences = df_sample['summary'].fillna('').values
y = df_sample['score']

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)

vect_dict = {'count':count_vect,'tfidf_word':tfidf_vect,'tfidf_ngram':tfidf_vect_ngram,'tfidf_ngram_chars':tfidf_vect_ngram_chars}

# count_vect.fit(sentences)
# tfidf_vect.fit(sentences)
# tfidf_vect_ngram.fit(sentences)
# tfidf_vect_ngram_chars.fit(sentences)

In [0]:
from sklearn.model_selection import train_test_split
# sentences_train, sentences_test, y_train, y_test = train_test_split(sentences,y)
def get_train_test(vectorizer,sentences,y):
  sentences_train, sentences_test, y_train, y_test = train_test_split(sentences,y)
  vectorizer.fit(sentences)
  X_train = vectorizer.transform(sentences_train)
  X_test  = vectorizer.transform(sentences_test)
  return X_train,X_test,y_train,y_test

In [0]:
X_train,X_test,y_train,y_test = get_train_test(count_vect,sentences,y)
X_test.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
from sklearn.linear_model import LinearRegression
clf = LogisticRegression()
clf.fit(X_train.toarray(),y_train)
y_pred = clf.predict(X_test.toarray())
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [0]:
#@title Regression Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


In [0]:
def regression_metrics(y_test,y_pred):
  mae = metrics.mean_absolute_error(y_test, y_pred)
  mse = metrics.mean_squared_error(y_test, y_pred)
  rms = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
  print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
  print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
  print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
  return mae,mse,rms

In [0]:
for vect_key in vect_dict:
  vect = vect_dict[vect_key]
  X_train,X_test,y_train,y_test = get_train_test(vect,sentences,y)
  

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'analyzer' != 'word'
  warnings.warn("The parameter 'token_pattern' will not be used"


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
